<a href="https://colab.research.google.com/gist/adawolfs/02684920abf29d96e707d6ca9aec644a/preprocesamiento_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN en Español

Este notebook esta destinado a servir como ejemplo practico para demostrar el funcionamiento de una red neuronal convolucional.

El ejemplo utilizado es meramente didactico, pero explica de buena manera los siguietes aspectos:

- Recolección de información
- Proprocesamiento de data
- Preparación de data de entrenamiento y de prueba
- Diseño de una Red Neuronal
- Entrenamiento de Red Neuronal
- Validación y test de el Modelo.

In [ ]:
#@title
!git clone https://github.com/adawolfs/CNN_en_espanol.git

# Resize de imagenes

Ya que la data debe de estar normalizada ejecutaremos un script que se encargara de hacer la primera manipulación de las imagenes para darles un tamaño estandard.


In [ ]:

%matplotlib inline

import matplotlib.pyplot as ptl # Utilizada para visualización de data
from PIL import Image # Utilizada para manipulación de Imagenes
from pathlib import Path # Utilizada para manipulación de archivos
import random # emmmm Random!
import shutil # Path tiene sus limitantes...

IMAGE_RESIZE_PX = 64

def print_image(img):
  ptl.imshow(img)
  ptl.axis('on')
  ptl.show()

def resize_images(source_dir, target_dir):
  """
  resize_images
  Esta función hace un resize de todas las imagenes en source_dir,
  y las almacena utilizando valores numericos dentro de target_dir

  se espera que source_dir tenga la siguiente estructura:
  source_dir/
   - dir01/
   -- image.jpg
   -- image1.jpg
   -- imagexx.jpg
   - dir02/
   -- image.jpg
   -- image1.jpg
   -- imagexx.jpg
   - dir03
   ...
  """
  source_dir = Path(source_dir)
  target_dir = Path(target_dir)

  try:
    target_dir.mkdir()
  except FileExistsError:
    print("Directorio destino ya existe.")
    return

  for dir in source_dir.iterdir():
    print(f"Directorio: {dir.name}")
    x = 0
    for image in dir.iterdir():
      #print(f"Resizing: {image.name}")
      img = Image.open(image)
      img = img.convert("RGB")
      
      if x<5 :
        print_image(img)

      img = img.resize((IMAGE_RESIZE_PX,IMAGE_RESIZE_PX))
      if x<5 :
        print_image(img)

      img.save(target_dir/f"{dir.name}_{x}.jpg")
      x = x+1
  

In [ ]:
!rm -rf CNN_en_espanol/data/resized

In [ ]:
 resize_images("CNN_en_espanol/data/original", "CNN_en_espanol/data/resized")

# Generación de imagenes 
Ya que los ejemplos son limitados utilizaremos el siguiente script para generar multiples variaciones.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
def generate_images(source_path, target_path, generations):
  """
  Este metodo generará imagees extra con distintas variaciones de las
  imagenes originales con la inteción de ayudar al modelo a generalizar y
  no hacer un sobre entrenamiento (overfitting) sobre la data
  """
  ## https://keras.io/api/preprocessing/image/#imagedatagenerator-class
  datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    channel_shift_range = 10,
    fill_mode= 'nearest'
  )
  source_path = Path(source_path)
  target_path = Path(target_path)
    
  if target_path.exists():
    print("Target already exists, please delete it.")
    return 0

  target_path.mkdir(parents=True)
      
  # Se hace una busqueda sobre todas las imagenes
  for image in source_path.iterdir():
    image_name, image_ext = image.name.split(".")
    img = load_img(image)
    img = img_to_array(img) # Se transforma al imagen en un array manipulable
    img = img.reshape((1,) + img.shape)
    #img = np.array([image])
    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir=target_path, save_prefix=image_name, save_format="jpg"):
      i = i+1
      if i >= generations:
        break

In [ ]:
!rm -rf CNN_en_espanol/data/generated

In [ ]:
generate_images("CNN_en_espanol/data/resized", "CNN_en_espanol/data/generated", 100)

In [ ]:
!echo "Imagenes orignales de entrenamiento: $(ls -la CNN_en_espanol/data/resized | wc -l)"
!echo "Imagenes generadas de entrenamiento: $(ls -la CNN_en_espanol/data/generated | wc -l)" 


# Cargar imagenes y sus respectivos labels


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from pathlib import Path
categories = ['bulbasaur', 'charmander', 'squirtle']

def img_set(images_path, test_size):    
    train_images = []
    train_labels = []

    test_images = []
    test_labels = []

    images_path = Path(images_path)
    
    for img in images_path.iterdir():
        image = load_img(img)
        image = img_to_array(image)
        label = categories.index(img.name.split('_')[0])
      
        if len(test_images) < test_size and random.randint(0,255) < 100:
          test_labels.append(label)
          test_images.append(image)
        else:
          train_labels.append(label)
          train_images.append(image)

    return np.array(train_images), np.array(train_labels), np.array(test_images), np.array(test_labels)

In [ ]:
x_train, y_train, x_test, y_test = img_set("CNN_en_espanol/data/generated", 20000)

print(f"Imagenes de entrenamiento: {y_train.shape}")
print(f"Imagenes de test: {y_test.shape}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout

X_train = np.array(x_train).astype('float32')
X_test = np.array(x_test).astype('float32')
X_train /= 255
X_test /= 255

num_classes=3
#Y_train = to_categorical(y_train, num_classes=num_classes)
#Y_test = to_categorical(y_test, num_classes=num_classes)
Y_train = y_train
Y_test = y_test

In [ ]:
Y_test.shape

In [ ]:
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=X_train.shape[1:]),
#     keras.layers.Dense(128, activation=tensorflow.nn.relu),
#     keras.layers.Dense(num_classes, activation=tensorflow.nn.softmax)
# ])

model = Sequential()
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
model.add(Conv2D(32, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.8))

model.add(Dense(num_classes, input_shape=[num_classes]))
model.add(Activation('softmax'))

#optimizer = optimizers.Adam()
#model.compile(loss='sparse_categorical_crossentropy',
#             optimizer=optimizer,
#             metrics=['accuracy'])

model.compile(optimizer='adam', 
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])



In [ ]:
epochs = 1
batch_size = 100
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

# Verificamos los valores de test

Haremos una validación de los valores de test

In [ ]:
from PIL import Image
from pathlib import Path
import numpy as np

fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(hspace=0.3, wspace=0.1)
rows = 10
cols = 10

# Haremos la verificación sobre las primeras 100 imagenes.
for i in range(0,100):
  image = x_test[i] # Leemos desde la data de prueba
  image_array =  np.array([image]) # Preparamos el array {}
  image = array_to_img(image) # 
  # Realizamos una predicción para esta imagen
  predictions = model.predict(image_array) 
  prediction = np.argmax(predictions, axis=1)
  ax = fig.add_subplot(rows, cols, i+1)
  ax.axis('Off')
  plt.imshow(image, interpolation=None)
  ax.set_title(categories[prediction[0]], fontsize=20)

In [ ]:
from PIL import Image
from pathlib import Path
import numpy as np
def recognize_image(image_path):
    """
    Esta funcion se encargara de tomar cualquier imagen
    transformarla a un formato entendible por nuestro modelo
    y obtener una predicción
    """
    image_path = Path(image_path)
    resized_image_path = image_path.parent/f"resized_{image_path.name}"
    img = Image.open(image_path)
    img = img.convert("RGB")
    img = img.resize((IMAGE_RESIZE_PX,IMAGE_RESIZE_PX))
    img.save(resized_image_path)
    
    image = load_img(resized_image_path)
    image = img_to_array(image)
    img_input_array = np.array([image])
    print(img_input_array.shape)
    predictions = model.predict(img_input_array)
    print(predictions)
    prediction = np.argmax(predictions, axis=1)
    ptl.imshow(img)
    ptl.axis('on')
    ptl.title(categories[prediction[0]])
    ptl.show()

In [ ]:
# Test Faciles
!wget -O test.jpg https://pbs.twimg.com/profile_images/761195323628875776/-I0GtKkL.jpg
#!wget -O test.jpg https://w7.pngwing.com/pngs/742/119/png-transparent-pokemon-x-and-y-pikachu-bulbasaur-pokemon-go-pikachu-carnivoran-grass-cartoon.png
#!wget -O test.jpg https://vignette.wikia.nocookie.net/es.pokemon/images/e/e3/Squirtle.png/revision/latest?cb=20160309230820

# Test dificiles
#!wget -O test.jpg https://i.ibb.co/ctFz4My/png-transparent-pokemon-x-and-y-pikachu-bulbasaur.png
#!wget -O test.jpg https://i.ytimg.com/vi/bRyn_aBAosM/sddefault.jpg
#!wget -O test.jpg https://i.vimeocdn.com/portrait/5305191_300x300
#!wget -O test.jpg  https://i.ibb.co/xqBV3SM/latest-cb-20160309230820.png
recognize_image("test.jpg")


# Exportar modelos

Una vez tengamos un modelo entrenado podemos exportarlo para volver a utilizarlo en el futuro, o en algun otro sistema.


In [ ]:

keras_file = 'pokemon_cnn.h5'
keras.models.save_model(model, keras_file)


In [ ]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("pokemon_cnn.tflite", "wb").write(tflite_model)